RRT Path Planning Algorithm

This notebook demonstrates the Rapidly-exploring Random Tree (RRT) algorithm for path planning in a 2D environment with obstacles.

The RRT algorithm is a powerful and widely used method for finding a collision-free path from a start point to a goal point in a complex, high-dimensional space. It works by incrementally building a tree from the start point, randomly exploring the environment, and growing the tree towards these random points.

__How RRT Works:__

1. __Initialize the Tree:__ The tree starts with just the start node.

2. __Iterative Expansion:__ In each iteration, the algorithm does the following:

3. __Sample a Point:__ A random point is sampled from the environment. With a small probability (`GOAL_SAMPLE_RATE`), the goal point is sampled to encourage growth towards the goal.

4. __Find Nearest Node:__ The nearest node in the existing tree to the sampled point is found.

5. __Steer:__ A new node is created a fixed distance (`STEP_SIZE`) from the nearest node, in the direction of the sampled point.

6. __Collision Check:__ The path from the nearest node to the new node is checked for collisions with obstacles.

7. __Add to Tree:__ If the path is collision-free, the new node is added to the tree, with its parent set to the nearest node.

8. __Goal Check:__ After adding the new node, its distance to the goal is checked. If it's within a certain threshold (`GOAL_THRESHOLD`), a path is considered found.

9. __Path Reconstruction:__ If a path is found, it is reconstructed by backtracking from the goal node to the start node using the parent pointers.

# 1. Setup and Initialization

First, we will import the necessary libraries and define the parameters for our RRT algorithm.

- `numpy` and `matplotlib.pyplot` for numerical operations and visualization.

- `random` and `math` for generating random numbers and mathematical functions.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# --- RRT Algorithm Parameters ---
# The maximum number of iterations to search for a path
MAX_ITERATIONS = 5000  
# The fixed distance for extending a new node from its nearest neighbor
STEP_SIZE = 15.0      
# The probability of sampling the goal point instead of a random point
GOAL_SAMPLE_RATE = 0.05 
# The distance threshold to consider the goal reached
GOAL_THRESHOLD = 5.0

# 2. Node and Helper Functions

We define a class `RRTNode` to represent the nodes of our tree and a few helper functions.

- `RRTNode`: Stores the `x`, `y` coordinates and a parent pointer to reconstruct the path.

- `distance:` Calculates the Euclidean distance between two nodes.

- `is_collision_free`: A function to check if a path segment between two nodes is clear of obstacles. We use a simple discretization method for this check.

- `find_nearest_node`: Iterates through the tree to find the node closest to a given point.

- `steer`: Generates a new node a fixed `STEP_SIZE` away from a `nearest_node` towards a `random_node`.

In [ ]:
class RRTNode:
    """Represents a node in the RRT tree."""
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.parent = None


def distance(node1, node2):
    """Calculates the Euclidean distance between two nodes."""
    return math.sqrt((node1.x - node2.x)**2 + (node1.y - node2.y)**2)


Checks if the path between two nodes collides with any obstacle using discretization

In [ ]:

def is_collision_free(start_node, end_node, obstacles, num_checks=20):
    """Checks if the path between two nodes collides with any obstacle using discretization."""
    for i in range(num_checks + 1):
        t = i / num_checks
        # Start your code here. 2 lines of code.
        x = None
        y = None
        # End your code here.
        for obs in obstacles:
            ox, oy, w, h = obs
            if ox <= x <= ox + w and oy <= y <= oy + h:
                return False
    return True


Finds the node in the tree closest to the random node.

In [ ]:

def find_nearest_node(tree, random_node):
    """Finds the node in the tree closest to the random node."""
    min_dist = float('inf')
    nearest_node = None
    # Start your code here. 5 lines of code.
    
    # End your code here.
    return nearest_node


In [ ]:

def steer(nearest_node, random_node, step_size):
    """Generates a new node a fixed distance towards the random node."""
    dist = distance(nearest_node, random_node)
    if dist < step_size:
        return random_node
    
    theta = math.atan2(random_node.y - nearest_node.y, random_node.x - nearest_node.x)
    new_x = nearest_node.x + step_size * math.cos(theta)
    new_y = nearest_node.y + step_size * math.sin(theta)
    return RRTNode(new_x, new_y)

# 3. Visualization Function

This function will help us visualize the RRT tree, the start and goal points, the obstacles, and the final path.

In [ ]:
def visualize_rrt(tree, start, goal, obstacles, path=None):
    """Visualizes the RRT tree, start, goal, and obstacles."""
    plt.figure(figsize=(10, 10))
    ax = plt.gca()

    # Plot obstacles
    for obs in obstacles:
        rect = plt.Rectangle((obs[0], obs[1]), obs[2], obs[3], color='gray', alpha=0.7)
        ax.add_patch(rect)

    # Plot RRT tree
    for node in tree:
        if node.parent:
            plt.plot([node.x, node.parent.x], [node.y, node.parent.y], '-g', linewidth=0.5)

    # Plot start and goal
    plt.plot(start.x, start.y, 'go', markersize=10, label='Start')
    plt.plot(goal.x, goal.y, 'ro', markersize=10, label='Goal')
    
    # Plot the final path if found
    if path:
        path_x = [node.x for node in path]
        path_y = [node.y for node in path]
        plt.plot(path_x, path_y, '-b', linewidth=2, label='Path')

    plt.title("Rapidly-exploring Random Tree (RRT) Visualization")
    plt.xlabel("X-coordinate")
    plt.ylabel("Y-coordinate")
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.show()

# 4. Main RRT Planning Algorithm

This is the core function that implements the main loop of the RRT algorithm. It returns the final tree and the found path.

In [ ]:
def rrt_planning(start, goal, obstacles, bounds):
    """Main RRT planning function."""
    tree = [start]
    path_found = False
    goal_node = None
    
    for i in range(MAX_ITERATIONS):
        # Sample a random point
        if random.random() < GOAL_SAMPLE_RATE:
            random_node = goal
        else:
            x = random.uniform(bounds[0], bounds[1])
            y = random.uniform(bounds[2], bounds[3])
            random_node = RRTNode(x, y)
            
        # Find nearest node in the tree
        nearest_node = find_nearest_node(tree, random_node)
        
        # Steer towards the random node
        new_node = steer(nearest_node, random_node, STEP_SIZE)
        
        # Check for collision
        if is_collision_free(nearest_node, new_node, obstacles):
            new_node.parent = nearest_node
            tree.append(new_node)
            
            # Check if goal is reached
            if distance(new_node, goal) < GOAL_THRESHOLD:
                goal_node = new_node
                path_found = True
                print("Path found!")
                break
                
    if not path_found:
        print("Path not found after", MAX_ITERATIONS, "iterations.")
        return tree, None

    # Reconstruct the path
    path = []
    current_node = goal_node
    while current_node is not None:
        path.append(current_node)
        current_node = current_node.parent
    path.reverse()
    
    return tree, path

# 5. Running the Simulation

Finally, let's define our environment with a start point, a goal point, and a set of rectangular obstacles. Then we run the `rrt_planning` function and visualize the results.

Feel free to modify the `start`, `goal`, and `obstacles` to test the algorithm with different scenarios.

In [ ]:
if __name__ == "__main__":
    # Define the environment
    bounds = (0, 100, 0, 100)  # x_min, x_max, y_min, y_max
    start = RRTNode(10, 10)
    goal = RRTNode(90, 90)
    
    obstacles = [
        (20, 20, 5, 50),
        (40, 50, 40, 5),
        (60, 20, 5, 50),
        (70, 70, 25, 5)
    ]
    
    # Run the RRT planning algorithm
    tree, path = rrt_planning(start, goal, obstacles, bounds)

    # Visualize the results
    visualize_rrt(tree, start, goal, obstacles, path)